In [1]:
from netin.models import DPAHModel

In [2]:
from multiprocessing import Pool, cpu_count
from typing import Dict, List, Set


from src.seed import (seed_degree, 
    seed_polarized, 
    seed_random, 
    seed_polarized_centrality_mixed, 
    seed_polarized_centrality, 
    seed_polarized_degree, 
    seed_centrality_mixed, 
    seed_centrality, 
    seed_influence_maximization
)


from evaluation import evaluate_all_seeds

This model connects a new node to the existing nodes biased by their group assignment. Homophily values of $h > 0.5$ indicate homophily, meaning that nodes prefer to connect to their own group. Heterophily is specified by $h < 0.5$. Nodes tend to connect to the other group.

Instead of the number of links per each new node, the directed graph models parameterize the density of the final network by the parameter $d$. Additionally, we need to define activity levels for outgoing links for both the minority and majority group as $plo_m/M$

In [3]:
N = 2_000 # Number of total nodes
f_m = 0.3 # Fraction of minority nodes
h = 0.9 # Homophilic
d = 0.01 # desity of the graph
plo = 2.1
#final number of edges will be: e = d * n (n-1)

In [4]:
# DPAH graphs:
# Homophilic h > 0.5
# Neutral h = 0.5
# Heterophilic h < 0.5

homophily_values = [0.1, 0.5, 0.9]

In [5]:
dpa_model = DPAHModel(N=N, f_m=f_m, d=d, plo_M=plo, plo_m=plo, h_M=h, h_m=h, seed=12345)
dpa_graph = dpa_model.simulate()

In [6]:
G = dpa_graph.to_nxgraph() # This creates a copy of the graph

for node, attr in G.nodes(data=True):
    attr['color'] = attr.pop('minority')

for node in G.nodes():
        in_degree = G.in_degree(node)
        for neighbor in G.successors(node):
            G[node][neighbor]["weight"] = 1 / (2 * in_degree) if in_degree > 0 else 1

In [7]:
seed_size = G.number_of_nodes() // 40
print(f"Seed size: {seed_size}")
k = seed_size
max_iter = 1000
budget = 2 * seed_size

Seed size: 50


In [8]:
# Define your seed functions
seed_functions = {
    "Random": seed_random,
    "MIA": lambda G, seed_size: seed_influence_maximization(G, seed_size, 500, 8, 0),
    "Degree": seed_degree,
    "Centrality": seed_centrality,
    "Centrality Mixed": seed_centrality_mixed,
    "Polarized": lambda G, seed_size: seed_polarized(G, seed_size, 0),
    "Polarized Degree": lambda G, seed_size: seed_polarized_degree(G, seed_size, 0),
    "Polarized Centrality": lambda G, seed_size: seed_polarized_centrality(G, seed_size, 0),
    "Polarized Centrality Mixed": lambda G, seed_size: seed_polarized_centrality_mixed(G, seed_size, 0),
}

# Example of how to call the function
all_results_df = evaluate_all_seeds(G, seed_functions, seed_size, k, max_iter, budget, "netin_09_2000")

# Display the combined results
print("Combined Results for All Seed Functions")
all_results_df

Running evaluation for seed function: Random
    Running evaluation for graph modification: Original Graph


100%|██████████| 1000/1000 [00:00<00:00, 436497.45it/s]


    Running evaluation for graph modification: Random


100%|██████████| 1000/1000 [00:03<00:00, 272.15it/s]


    Running evaluation for graph modification: Degree


...


100%|██████████| 1000/1000 [00:04<00:00, 233.77it/s]

    Running evaluation for graph modification: Custom V2



100%|██████████| 1000/1000 [00:03<00:00, 257.31it/s]

Combined Results for All Seed Functions


Metric,Seed Function,Graph Modification,Avg Activated Nodes,Activated Nodes Std Dev,"Avg Activated Nodes, Color (0)","Activated Nodes Std Dev, Color (0)","Avg Activated Nodes, Color (1)","Activated Nodes Std Dev, Color (1)",Avg Color Activation Count,Color Activation Count Std Dev,Number of Nodes,Number of Edges
0,Centrality,Original Graph,66.762,8.321,64.533,7.607,2.229,1.710,1.030,1.307,2000.0,39980.0
1,Centrality,PrefAtt,2000.000,0.000,1398.000,0.000,602.000,0.000,1209.008,6.505,2000.0,40080.0
2,Centrality,Jaccard,2000.000,0.000,1398.000,0.000,602.000,0.000,2259.527,684.870,2000.0,40080.0
3,Centrality,Degree,2000.000,0.000,1398.000,0.000,602.000,0.000,606.192,5.314,2000.0,40080.0
4,Centrality,TopK,2000.000,0.000,1398.000,0.000,602.000,0.000,1221.404,7.742,2000.0,40080.0
...,...,...,...,...,...,...,...,...,...,...,...,...
76,Random,TopK,2000.000,0.000,1398.000,0.000,602.000,0.000,623.635,13.901,2000.0,40080.0
77,Random,KKT,2000.000,0.000,1398.000,0.000,602.000,0.000,586.122,6.942,2000.0,40080.0
78,Random,Random,204.016,25.207,139.245,17.861,64.771,11.395,41.614,9.398,2000.0,40080.0
79,Random,Custom,200.427,20.578,119.154,15.127,81.273,9.547,50.537,8.027,2000.0,40080.0


In [9]:
#all_results_df.to_csv(f"test.csv", index=False)
all_results_df.to_csv(f"results/results_netin_{N}_nodes_{h}_hvalue_{d}_density_{plo}_plo_1.csv", index=False)